In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="input/"
file_name="my-m_ee_Run14AuAu_56th_new_19668_702runs.root"#m_ee_Run14AuAu_56th_new_19668_404runs 20th_QA_19460_all
part_name = ["e^{+}","e^{-}"]
hist_psi = ["BBC_psi_hist","FVTX_psi_hist"]
hist_cos = ["cos_BBC_hist","cos_FVTX_hist"]
hist_v2 = ["v2_BBC_hist","v2_FVTX_hist"]
hists_all = [hist_psi,hist_cos,hist_v2]
N_type = len(hists_all)

In [4]:
hists_read = []

infile = root.TFile.Open(file_path+file_name, "read")
for itype in range(N_type):
    hist_select = []
    for ihist in range(len(hists_all[itype])):
        hist_select.append(infile.Get(hists_all[itype][ihist]))
        hist_select[-1].SetDirectory(root.nullptr)
        #hist_select[-1].SetName(hist_select[-1].GetName()+f"{itype}")
    hists_read.append(hist_select)

infile.Close()

In [5]:
type_loc = 0
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,N_centr)
legends=[]
for itype in range(N_type_loc):
    for icentr in range(N_centr):
        c0.cd(itype+icentr*N_type_loc+1)
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj = hists_read[type_loc][itype].ProjectionX(f"psi_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),3,3)
        Format_Hist_total(proj,title_x="#psi, rad",  title_y="counts", left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        proj.Draw()
        legends.append(Draw_Legend_Title(0.2,0.1,0.8,0.7,f"{psi_names[itype]}, {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
        legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [6]:
type_loc = 1
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,500)
c0.Divide(N_type_loc,1)
legends=[]
graphs = []
h = []
resoluton = []
for itype in range(N_type_loc):
    resoluton_type = []
    graphs.append(root.TGraphErrors(N_centr))
    c0.cd(itype+1)
    h.append(Format_Pad_old( left=0,  right=100,  min_boarder=0,  max_boarder=0.9,  title_x="centrality",  title_y="RES{#Psi}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    for icentr in range(N_centr):
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj0 = hists_read[type_loc][itype].ProjectionX(f"psi0_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),1,1)
        proj1 = hists_read[type_loc][itype].ProjectionX(f"psi1_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),2,2)
        proj2 = hists_read[type_loc][itype].ProjectionX(f"psi2_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),3,3)
        mean0, mean1, mean2 = proj0.GetMean(), proj1.GetMean(), proj2.GetMean()
        mean_err0, mean_err1, mean_err2 = proj0.GetMeanError(), proj1.GetMeanError(), proj2.GetMeanError()
        x = (central_bins[2*icentr]+central_bins[2*icentr+1])/2
        x_err = 2
        res = ( mean0 * mean1 / mean2 )**0.5
        resoluton_type.append(res)
        res_err = res * ( (mean_err0/mean0)**2 + (mean_err1/mean1)**2 + (mean_err2/mean2)**2  )**0.5
        graphs[-1].SetPoint(icentr,x, res)
        graphs[-1].SetPointError(icentr,x_err,res_err)
        print(itype,icentr,mean0, mean1, mean2)
    resoluton.append(resoluton_type)
    Format_Graph( graphs[-1],  Mstyle=20+itype,  Msize=1.4,  Mcolor=1+itype,  Lwidth=1,  Lcolor=1+itype,  Malpha=1,  Lalpha=1)
    graphs[-1].Draw("same P")
    legends.append(Draw_Legend_Title(0.6,0.5,0.8,0.9,f"{psi_names[itype]}",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2
0 0 0.17637960203481073 0.17616957508176992 0.25803362919350437
0 1 0.15618642793834764 0.1554911298894522 0.2278523492102321
0 2 0.2349114721453379 0.23491679314723693 0.3397472120307911
0 3 0.13652114336233326 0.13773535914430815 0.2122705878109841
0 4 0.03356154606764346 0.03451503513723704 0.057653077005964365
1 0 0.1593618957883394 0.15868701909409239 0.0560701014699307
1 1 0.14164763215376622 0.14124844113496005 0.04929424248549185
1 2 0.21014880870466585 0.20912883110168526 0.07744309604683741
1 3 0.12639754470778503 0.12522506238818096 0.0377923717707631
1 4 0.03266707993354762 0.032728847132052166 0.007275839578520892


In [7]:
type_loc = 2
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c2",f"c2",1400,400*N_centr)
c0.Divide(N_type_loc,N_centr)
legends=[]
h = []
myexps = []
for itype in range(N_type_loc):
    myexps_type = []
    for icentr in range(N_centr):
        c0.cd(itype+icentr*N_type_loc+1)
        graphs.append(root.TGraphErrors(N_centr))
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj = hists_read[type_loc][itype].ProjectionY(f"pt_{itype}_{icentr}",1,200,int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5))
        Format_Hist_total(proj,title_x="p_{T}, GeV",  title_y="counts", left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        proj.Draw()
        myexps_type.append(root.TF1(f"myexp_{itype}_{icentr}","([0]*0.1*([1]-1)*([1]-2))*(pow((sqrt(x*x+[3]*[3])+[2]-[3])/([2]),-[1]))/(2*3.14159)/([2]+[3]*([1]-2))/([2]+[3])",0.2,5))
        myexps_type[-1].SetParameter(3,0.135)
        myexps_type[-1].SetParameter(0,proj.GetMaximum()*12)
        myexps_type[-1].SetParameter(1,15)
        myexps_type[-1].SetParameter(2,4.7)
        proj.Fit(myexps_type[-1],"Q","",1.3,5)
  
        root.gPad.SetLogy()
        legends.append(Draw_Legend_Title(0.2,0.1,0.8,0.7,f"{psi_names[itype]}, {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
        legends[-1].Draw()
    myexps.append(myexps_type)

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [8]:
pt = []
pt_err = []
N_pt = len(pt_boarders) - 1
N_steps = 100
for itype in range(len(hists_all[2])):
    pt_itype = []
    pt_err_itype = []
    for icentr in range(N_centr):
        pt_icentr = []
        pt_err_icentr = []
        for i in range(N_pt):
            pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(0+0.5)
            for istep in range(N_steps):
                pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(istep+0.5)
                if myexps[itype][icentr].Integral(pt_boarders[i],pt_step,1e-15) > myexps[itype][icentr].Integral(pt_step,pt_boarders[i+1],1e-15):
                    break
            pt_icentr.append( round(pt_step,4))
            pt_err_icentr.append((pt_boarders[i+1]-pt_boarders[i])/100*2)
        pt_itype.append(pt_icentr)
        pt_err_itype.append(pt_err_icentr)
    pt.append(pt_itype)
    pt_err.append(pt_err_itype)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  [[[0.4445, 0.5445, 0.6445, 0.781, 0.983, 1.2125, 1.5675, 2.101, 2.785, 3.8825], [0.4445, 0.5445, 0.6455, 0.781, 0.983, 1.2155, 1.5675, 2.101, 2.785, 3.8825], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5625, 2.101, 2.785, 3.8975], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5675, 2.101, 2.795, 3.9125], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5675, 2.101, 2.795, 3.9125]], [[0.4445, 0.5445, 0.6445, 0.781, 0.983, 1.2125, 1.5675, 2.101, 2.785, 3.8825], [0.4445, 0.5445, 0.6455, 0.781, 0.983, 1.2155, 1.5675, 2.101, 2.785, 3.8825], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5625, 2.101, 2.785, 3.8975], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5675, 2.101, 2.795, 3.9125], [0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5675, 2.101, 2.795, 3.9125]]] 
pt_boarders =  [0.4, 0.5, 0.6, 0.7, 0.9, 1.1, 1.4, 1.9, 2.5, 3.5, 5.0]


In [9]:
type_loc = 2
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,3)
legends=[]
graphs = []
h = []
for icentr in range(N_centr):
    c0.cd(icentr+1)
    h.append(Format_Pad_old( left=0,  right=5,  min_boarder=0,  max_boarder=0.4,  title_x="p_{T} [GeV/c]",  title_y="v_{2}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    legends.append(Draw_Legend_Title(0.25,0.6,0.75,0.8,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].SetNColumns(3)
    for itype in range(N_type_loc):
        for ipt in range(N_pt):
            graphs.append(root.TGraphErrors(N_pt))
            v2_proj = hists_read[type_loc][itype].ProjectionX(f"v2_{itype}_{icentr}_{ipt}",int(pt_boarders[ipt]*10)+1,int(pt_boarders[ipt+1]*10)\
                                                            ,int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5))
            v2 = v2_proj.GetMean()/resoluton[itype][icentr]
            v2_err = v2_proj.GetMeanError()/resoluton[itype][icentr]
            x_pt = pt[itype][icentr][ipt]
            x_pt_err = pt_err[itype][icentr][ipt]
            graphs[-1].SetPoint(icentr,x_pt, v2)
            graphs[-1].SetPointError(icentr,x_pt_err,v2_err)
            Format_Graph( graphs[-1],  Mstyle=20+itype,  Msize=1.2,  Mcolor=1+itype,  Lwidth=1,  Lcolor=1+itype,  Malpha=1,  Lalpha=1)
            graphs[-1].Draw("same P")
        legends[-1].AddEntry(graphs[-1],psi_names[itype],"lep")
    legends[-1].Draw()
    
c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [10]:
type_loc = 2
psi_choice = 1
part_names = ["e^{+}","e^{-}"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,3)
legends=[]
graphs = []
h = []
for icentr in range(N_centr):
    c0.cd(icentr+1)
    h.append(Format_Pad_old( left=0,  right=5,  min_boarder=0,  max_boarder=0.4,  title_x="p_{T} [GeV/c]",  title_y="v_{2}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    legends.append(Draw_Legend_Title(0.2,0.6,0.9,0.8,f"FVTX {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].SetNColumns(3)
    for ipart in range(2):
        for ipt in range(N_pt):
            graphs.append(root.TGraphErrors(N_pt))
            v2_proj = hists_read[type_loc][psi_choice].ProjectionX(f"v2_charge_{ipart}_{icentr}_{ipt}",int(pt_boarders[ipt]*10)+1,int(pt_boarders[ipt+1]*10)\
                                                            ,int(central_bins[2*icentr]/5+20*ipart)+1,int(central_bins[2*icentr+1]/5+20*ipart))
            v2 = v2_proj.GetMean()/resoluton[psi_choice][icentr]
            v2_err = v2_proj.GetMeanError()/resoluton[psi_choice][icentr]
            x_pt = pt[psi_choice][icentr][ipt]
            x_pt_err = pt_err[psi_choice][icentr][ipt]
            graphs[-1].SetPoint(icentr,x_pt, v2)
            graphs[-1].SetPointError(icentr,x_pt_err,v2_err)
            Format_Graph( graphs[-1],  Mstyle=20+ipart,  Msize=1.2,  Mcolor=1+ipart,  Lwidth=1,  Lcolor=1+ipart,  Malpha=1,  Lalpha=1)
            graphs[-1].Draw("same P")
        legends[-1].AddEntry(graphs[-1],part_names[ipart],"lep")
    legends[-1].Draw()
    
c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2
